In [1]:
import json
import numpy as np

<h3>Functions<h3>

removes nodes that belong to a single road

In [2]:
def remove_singlepoints(intersections):
    keys = intersections.keys()
    for el in list(keys):
        if len(intersections[el]) == 1:
            del intersections[el]
    return intersections

removes ways that have less than 2 points => dead-end

In [3]:
def remove_dead_ends(roads, intersections):
    ways_with_only_intercections = {}
    imp_nodes = intersections.keys()
    possible_outliers = []
    for el in roads:
        temp = []
        for nod in roads[el]:
            if nod in imp_nodes:
                temp.append(nod)
        if len(temp) > 1:
            ways_with_only_intercections[el] = temp
        elif (len(temp) > 0):
            possible_outliers.append([temp[0],el])
    return ways_with_only_intercections, possible_outliers

removes ways from a nodes list:  { node: [way1, way2, ...] } \
possible_outliers contains [node, way] pairs

In [4]:
def remove_outliers(possible_outliers, nodes):
    for el in possible_outliers:
        nodes[el[0]].remove(el[1])
    return nodes

<h3>Loading data<h3>

In [5]:
f = open('ful2.json')
  
# returns JSON object as 
# a dictionary
data = json.load(f)

In [6]:
elements = data.get('elements')
elements[0].get('type')

'node'

In [7]:
nodes = np.asarray([[1,1,1]])
for el in elements:
    if el.get('type') == 'node':
        nodes = np.append(nodes, [[el.get('id'),el.get('lat'), el.get('lon')]], axis=0)

In [8]:
roads = {}
for el in elements:
    if el.get('type') == 'way':
        roads[el.get('id')] = el.get('nodes')

In [9]:
road_ids = np.asarray(list(roads.keys()))
node_intersections = {}

In [10]:

for el in roads:
    for nod in roads[el]:
        if nod in node_intersections:
            node_intersections[nod].append(el)
        else:
            node_intersections[nod] = [el]


<h3>Treatment<h3>

In [11]:
# remove all nodes that aren't on an intersection <=> they only belong to one way
node_intersections = remove_singlepoints(node_intersections)

In [12]:
# Removes all empty ways and ways with 1 node
# returns cleaned ways: only ways with at least 2 valid nodes
# returns possible outliers which are nodes that belong to a dead end
ways_with_only_intercections, possible_outliers = remove_dead_ends(roads, node_intersections)

In [13]:
# use previously found outliers to remove ways from the intersection list
node_intersections = remove_outliers(possible_outliers, node_intersections)

In [14]:
# since ways were removed we need to make sure to only keep valid intersections
node_intersections = remove_singlepoints(node_intersections)

In [15]:
# repeat the operation using the reduced way- and intersection- lists
ways_with_only_intercections, possible_outliers = remove_dead_ends(ways_with_only_intercections, node_intersections)

In [16]:
counter = 1
while (counter != 0):
    ways_with_only_intercections, possible_outliers = remove_dead_ends(ways_with_only_intercections, node_intersections)
    counter = len(possible_outliers)
    node_intersections = remove_outliers(possible_outliers, node_intersections)
    node_intersections = remove_singlepoints(node_intersections)

In [17]:
node_intersections

{30895600: [4212270, 560273013, 729995781],
 3119898379: [4212270, 306966911],
 24959733: [4212272, 134409265, 336687868],
 145711: [4212272, 25878856],
 1589527895: [4823038, 307095350, 307095354],
 30295321: [4823038, 307095356],
 65850052: [4823068, 310013142, 310013149],
 6511596502: [4823068, 333790813],
 3336442788: [4823074, 218878351, 319960411],
 3336442789: [4823074, 370724417],
 5328246393: [4823081, 551746152],
 5328246394: [4823081, 975462567],
 30895484: [4823083, 332241477, 336687868],
 3151978239: [4823083, 309856474],
 31281250: [4852437, 187008022, 307351131, 364411098],
 31281245: [4852437, 139431863],
 1528622890: [4852437, 139431860],
 3124579584: [4852437, 307349371],
 2013119764: [4891466, 190676489],
 5328246404: [4891466, 551746158],
 145725: [4917875, 59743150, 307362110],
 3124717674: [4917875, 307362130],
 1397783386: [5094133, 284762148, 284762151],
 2884677145: [5094133, 284762169],
 2384975497: [5182700, 229957653, 307095349],
 2384975500: [5182700, 30709

In [18]:
nodes_dict = {}
for node in nodes:
    nodes_dict[int(node[0])] = [node[1],node[2]]

In [19]:
import geopy.distance

def coord_dist(point1, point2):
    lat1, lon1 = nodes_dict[point1]
    lat2, lon2 = nodes_dict[point2]
    
    return geopy.distance.geodesic((lat1, lon1),(lat2, lon2)).km


In [20]:
# function returns a dict with all distances between nodes and endpoint

def astar_dist_maker(end):
    astar_dict = dict.fromkeys(node_intersections.keys())
    for i in astar_dict:
        astar_dict[i] = coord_dist(end, i)
    return astar_dict

In [21]:
import itertools

nodelist = dict.fromkeys(node_intersections.keys())
for el in nodelist:
    nodelist[el] = []

waysways_with_only_intercections = dict(itertools.islice(ways_with_only_intercections.items(),25))


for el in ways_with_only_intercections:
    j = len(ways_with_only_intercections[el])
    for i in range(0,j):
        if i == 0:
            nodelist[ways_with_only_intercections[el][i]].append([ways_with_only_intercections[el][i+1], coord_dist(ways_with_only_intercections[el][i], ways_with_only_intercections[el][i+1])])
        elif i == (j - 1):
            nodelist[ways_with_only_intercections[el][i]].append([ways_with_only_intercections[el][i-1], coord_dist(ways_with_only_intercections[el][i], ways_with_only_intercections[el][i-1])])
        else:
            nodelist[ways_with_only_intercections[el][i]].append([ways_with_only_intercections[el][i+1], coord_dist(ways_with_only_intercections[el][i], ways_with_only_intercections[el][i+1])])
            nodelist[ways_with_only_intercections[el][i]].append([ways_with_only_intercections[el][i-1], coord_dist(ways_with_only_intercections[el][i], ways_with_only_intercections[el][i-1])])

In [22]:
nodelist

{30895600: [[3119898379, 1.5889365528293868],
  [1589527813, 0.3132472954014613],
  [24959734, 0.8452257345903857]],
 3119898379: [[30895600, 1.5889365528293868], [36139992, 0.06753347886504338]],
 24959733: [[145711, 0.47865039989715824],
  [1477914943, 0.24319159910820698],
  [30895484, 0.5227564627383561]],
 145711: [[24959733, 0.47865039989715824], [282288486, 0.11970505758422804]],
 1589527895: [[30295321, 0.01090062183439088],
  [3121590084, 0.06004779054111978],
  [676827608, 0.08423878288846424]],
 30295321: [[1589527895, 0.01090062183439088],
  [3121590086, 0.06975952793638308]],
 65850052: [[6511596502, 0.03811223479282381],
  [3153697175, 0.04148258437254308],
  [65850041, 0.0383090935720886]],
 6511596502: [[65850052, 0.03811223479282381],
  [10760808131, 0.018002394482742753]],
 3336442788: [[3336442789, 0.005725745664393139],
  [3336444295, 0.006047143108054236],
  [2280858814, 0.07585780328056604]],
 3336442789: [[3336442788, 0.005725745664393139],
  [2302599593, 0.00831

In [23]:
import matplotlib.pyplot as plt
lattitudes = []
longitudes = []

points = list(node_intersections.keys())
count = len(points)
current = 1
for nod in points:
    lattitudes.append(nodes_dict[nod][0])
    longitudes.append(nodes_dict[nod][1])
    print(str(current) + "/" + str(count))
    current += 1

        

1/21188
2/21188
3/21188
4/21188
5/21188
6/21188
7/21188
8/21188
9/21188
10/21188
11/21188
12/21188
13/21188
14/21188
15/21188
16/21188
17/21188
18/21188
19/21188
20/21188
21/21188
22/21188
23/21188
24/21188
25/21188
26/21188
27/21188
28/21188
29/21188
30/21188
31/21188
32/21188
33/21188
34/21188
35/21188
36/21188
37/21188
38/21188
39/21188
40/21188
41/21188
42/21188
43/21188
44/21188
45/21188
46/21188
47/21188
48/21188
49/21188
50/21188
51/21188
52/21188
53/21188
54/21188
55/21188
56/21188
57/21188
58/21188
59/21188
60/21188
61/21188
62/21188
63/21188
64/21188
65/21188
66/21188
67/21188
68/21188
69/21188
70/21188
71/21188
72/21188
73/21188
74/21188
75/21188
76/21188
77/21188
78/21188
79/21188
80/21188
81/21188
82/21188
83/21188
84/21188
85/21188
86/21188
87/21188
88/21188
89/21188
90/21188
91/21188
92/21188
93/21188
94/21188
95/21188
96/21188
97/21188
98/21188
99/21188
100/21188
101/21188
102/21188
103/21188
104/21188
105/21188
106/21188
107/21188
108/21188
109/21188
110/21188
111/2118

In [24]:
max = 0
for nod in nodelist:
    if len(nodelist[nod]) > max:
        max = len(nodelist[nod])
        print(nod)
max

30895600
31281250
5327959450
1358274583


8

In [25]:
# primary = nodelist[31281250]
# previous = []
# for i in  range(0,15):
#     if primary[0] not in previous:
#         previous.append(primary[0])
#         print(primary[0])
#         primary = nodelist[primary[0]]
        
        
#     elif primary[1] not in previous:
#         previous.append(primary[1])
#         print(primary[1])
#         primary = nodelist[primary[1]]
        
        
#     else:
#         if len(primary) > 2:
#             print(primary[3])
#             previous.append(primary[3])

In [26]:
# f = open('shops.json')
  
# # returns JSON object as 
# # a dictionary
# shop = json.load(f)

In [27]:
# node = []
# way = []
# rel = []

# for el in shop["elements"]:
#     if el['type'] == 'node' and "tags" in el:
#         node.append([el['id'],el['lat'],el['lon']])


In [28]:
def openjson(file):
    f = open(file)
    data = json.load(f)
    node = []

    for el in data["elements"]:
        if el['type'] == 'node' and "tags" in el:
            node.append([el['id'],el['lat'],el['lon']])

    return node

In [29]:
shops = openjson("shops.json")
phys = openjson("phys.json")
sport = openjson("sport.json")
food = openjson("food.json")
amusement = openjson("amusement.json")
accomo = openjson("accomo.json")
hist = openjson("hist.json")

In [30]:
shops

[[311986274, 51.078223, 3.7022541],
 [314251389, 51.0454842, 3.7300126],
 [510812967, 51.0566899, 3.7250032],
 [686401512, 51.0622837, 3.751865],
 [686401578, 51.0641809, 3.7519413],
 [707141891, 51.0727392, 3.7110128],
 [761164858, 51.0224957, 3.7771323],
 [761164863, 51.0224341, 3.7772034],
 [761164867, 51.0223105, 3.7773462],
 [761164872, 51.0225536, 3.7770655],
 [761164880, 51.0226188, 3.7769902],
 [830298004, 51.0267048, 3.7109325],
 [895727805, 51.0569717, 3.7215632],
 [1081999323, 51.0414187, 3.7298355],
 [1096072983, 51.046571, 3.6565428],
 [1150961878, 51.0347978, 3.7256754],
 [1186301482, 51.0571656, 3.7247608],
 [1186322716, 51.0573645, 3.7250558],
 [1186342681, 51.057389, 3.7250906],
 [1186342684, 51.0575854, 3.7255667],
 [1186945842, 51.0549519, 3.7223202],
 [1186970669, 51.0570829, 3.7267536],
 [1186970689, 51.0570011, 3.726924],
 [1186972733, 51.056943, 3.7270152],
 [1186972734, 51.0569624, 3.7269655],
 [1186981126, 51.0568477, 3.7271305],
 [1186981129, 51.0568806, 3.727

In [31]:
def weights(nodes,type,node_weights):
    nods = list(node_intersections.keys())
    chosen = []
    for el in nodes:
    
        min = 10000000.0
        nodl = None
        lat = el[1]
        lon = el[2]
        for nod in nods:
            dist = abs(lat-nodes_dict[nod][0]) + abs(lon-nodes_dict[nod][1])
            if dist < min:
                min = dist
                nodl = nod
        chosen.append(nodl)
    for nod in node_intersections.keys():
        node_weights[nod][type] = 0
    for shop in chosen:
        node_weights[shop][type] += 1
    print(type + " done")
    return node_weights

In [32]:
node_weights = {}
for nod in node_intersections.keys():
    node_weights[nod] = {}

node_weights = weights(shops,'shop',node_weights)
node_weights = weights(phys,'phys',node_weights)
node_weights = weights(sport,'sport',node_weights)
node_weights = weights(food,'food',node_weights)
node_weights = weights(amusement,'amusement',node_weights)
node_weights = weights(accomo,'accomo',node_weights)
node_weights = weights(hist,'hist',node_weights)
node_weights


shop done
phys done
sport done
food done
amusement done
accomo done
hist done


{30895600: {'shop': 0,
  'phys': 0,
  'sport': 0,
  'food': 0,
  'amusement': 0,
  'accomo': 0,
  'hist': 0},
 3119898379: {'shop': 0,
  'phys': 0,
  'sport': 0,
  'food': 0,
  'amusement': 0,
  'accomo': 0,
  'hist': 0},
 24959733: {'shop': 0,
  'phys': 0,
  'sport': 0,
  'food': 0,
  'amusement': 0,
  'accomo': 0,
  'hist': 0},
 145711: {'shop': 0,
  'phys': 0,
  'sport': 0,
  'food': 0,
  'amusement': 0,
  'accomo': 0,
  'hist': 0},
 1589527895: {'shop': 0,
  'phys': 0,
  'sport': 0,
  'food': 0,
  'amusement': 0,
  'accomo': 0,
  'hist': 0},
 30295321: {'shop': 0,
  'phys': 0,
  'sport': 0,
  'food': 0,
  'amusement': 0,
  'accomo': 0,
  'hist': 0},
 65850052: {'shop': 0,
  'phys': 0,
  'sport': 0,
  'food': 0,
  'amusement': 0,
  'accomo': 0,
  'hist': 0},
 6511596502: {'shop': 0,
  'phys': 0,
  'sport': 0,
  'food': 0,
  'amusement': 0,
  'accomo': 0,
  'hist': 0},
 3336442788: {'shop': 0,
  'phys': 0,
  'sport': 0,
  'food': 0,
  'amusement': 0,
  'accomo': 0,
  'hist': 0},
 333

In [33]:
# jobj = json.dumps(node_weights)
# print(jobj)
# with open('result.json', 'w') as fp:
#     json.dump(jobj, fp)


In [34]:
# nods = list(node_intersections.keys())
# shops = []
# count = 0
# for el in node:
    
#     min = 10000000.0
#     nodl = None
#     lat = el[1]
#     lon = el[2]
#     for nod in nods:
#         dist = abs(lat-nodes_dict[nod][0]) + abs(lon-nodes_dict[nod][1])
#         if dist < min:
#             min = dist
#             nodl = nod
#     shops.append(nodl)
#     count += 1
#     print(count)
        

In [35]:
# node_weights = {}
# for nod in node_intersections.keys():
#     node_weights[nod] = {}
#     node_weights[nod]['shop'] = 0

In [36]:
node_weights

{30895600: {'shop': 0,
  'phys': 0,
  'sport': 0,
  'food': 0,
  'amusement': 0,
  'accomo': 0,
  'hist': 0},
 3119898379: {'shop': 0,
  'phys': 0,
  'sport': 0,
  'food': 0,
  'amusement': 0,
  'accomo': 0,
  'hist': 0},
 24959733: {'shop': 0,
  'phys': 0,
  'sport': 0,
  'food': 0,
  'amusement': 0,
  'accomo': 0,
  'hist': 0},
 145711: {'shop': 0,
  'phys': 0,
  'sport': 0,
  'food': 0,
  'amusement': 0,
  'accomo': 0,
  'hist': 0},
 1589527895: {'shop': 0,
  'phys': 0,
  'sport': 0,
  'food': 0,
  'amusement': 0,
  'accomo': 0,
  'hist': 0},
 30295321: {'shop': 0,
  'phys': 0,
  'sport': 0,
  'food': 0,
  'amusement': 0,
  'accomo': 0,
  'hist': 0},
 65850052: {'shop': 0,
  'phys': 0,
  'sport': 0,
  'food': 0,
  'amusement': 0,
  'accomo': 0,
  'hist': 0},
 6511596502: {'shop': 0,
  'phys': 0,
  'sport': 0,
  'food': 0,
  'amusement': 0,
  'accomo': 0,
  'hist': 0},
 3336442788: {'shop': 0,
  'phys': 0,
  'sport': 0,
  'food': 0,
  'amusement': 0,
  'accomo': 0,
  'hist': 0},
 333

In [37]:
nodelist[6931617487]

[[6931617492, 0.0036070450572690716], [6931617498, 0.0016060415003145345]]

In [38]:
start = 36727925
end = 246553575

In [39]:
import heapq

def astar(graph, start, end, nodecost):
    """
    A* algorithm implementation.

    Arguments:
    graph -- dictionary that holds direct neighbours of each node with their distances.
    start -- starting node.
    end -- target node.
    nodecost -- dictionary that holds the heuristic cost of reaching the end node from each node.

    Returns:
    Path from the starting node to the end node if found, otherwise None.
    """

    # Create a priority queue for open nodes.
    open_set = [(0, start)]
    # Create a dictionary to store the cost of the cheapest path to each node.
    g = {start: 0}
    # Create a dictionary to store the parent of each node.
    parent = {}

    while open_set:
        # Get the node with the lowest cost from the priority queue.
        current_cost, current_node = heapq.heappop(open_set)

        if current_node == end:
            # Construct the path by following the parent pointers.
            path = [end]
            while path[-1] != start:
                path.append(parent[path[-1]])
            path.reverse()
            walking_dist = 0
            for i in range(len(path)-1):
                temp_dict = dict(graph[path[i]])
                walking_dist += float(temp_dict[path[i+1]])
            return path, walking_dist

        # Loop through the neighbours of the current node.
        for neighbour, cost in graph[current_node]:
            # Calculate the tentative cost of reaching the neighbour.
            tentative_g = g[current_node] + cost
            # If this is a new node, or the new path is cheaper than the old one, update the cost and parent.
            if neighbour not in g or tentative_g < g[neighbour]:
                g[neighbour] = tentative_g
                f = tentative_g + nodecost[neighbour]
                heapq.heappush(open_set, (f, neighbour))
                parent[neighbour] = current_node

    # If we reach here, there is no path.
    return None

In [40]:
distances = astar_dist_maker(end)
path = astar(nodelist,start,end,distances)

In [41]:
# path, coords, distances

def pathmaker(distances, start, end, nodelist, increase):
    path, walking_dist = astar(nodelist,start,end,distances)
    coordinates = []
    for i in path:
        coordinates.append(nodes_dict[i])
    for el in path[1:-2]:
        distances[el] += increase

    return path, coordinates, distances, (str(round(walking_dist, 3)) + " km")

In [42]:
def scorecalc(path):
    score = {
        'shop': 0,
        'phys': 0,
        'sport': 0,
        'food': 0,
        'amusement': 0,
        'accomo': 0,
        'hist': 0
    }
    for node in path:
        score['shop'] += node_weights[node]['shop']
        score['phys'] += node_weights[node]['phys']
        score['sport'] += node_weights[node]['sport']
        score['food'] += node_weights[node]['food']
        score['amusement'] += node_weights[node]['amusement']
        score['accomo'] += node_weights[node]['accomo']
        score['hist'] += node_weights[node]['hist']
    return score

def shortscore(path):
    score = scorecalc(path)
    total = 0
    for i in score:
        total += score[i]
    return total


In [43]:
colors = [
    'red',
    'blue',
    'gray',
    'darkred',
    'orange',
    'beige',
    'green',
    'darkgreen',
    'lightgreen',
    'darkblue',
    'lightblue',
    'purple',
    'darkpurple',
    'pink',
    'cadetblue',
    'lightgray',
    'black',
    'lightred'
]

In [44]:
import folium

increase = 1.5

start2 = 2351687226
end2 = 600508280

distances = astar_dist_maker(end2)

number_of_paths = 14
output = []
for i in range(14):
    path, coordinates, distances, walking_dist = pathmaker(distances, start2, end2, nodelist, increase)
    output.append([path, coordinates, walking_dist])

m = folium.Map(location=coordinates[0], zoom_start=15)
folium.Marker(nodes_dict[start2]).add_to(m)
folium.Marker(nodes_dict[end2]).add_to(m)

color = 0
for i in output:
    folium.PolyLine(locations=i[1], color=colors[color], tooltip=i[2]).add_to(m)
    color += 1

# Display the map
m

In [45]:
import pandas as pd
table = pd.DataFrame(columns=['pathNumber', 'distance', 'score'])

nb = 1
for i in output:
    table.loc[nb] = ['path ' + str(nb), i[2], shortscore(i[0])]
    nb += 1
table

,pathNumber,distance,score
1,path 1,8.964 km,37
2,path 2,9.721 km,19
3,path 3,11.041 km,191
4,path 4,10.326 km,22
5,path 5,10.225 km,176
6,path 6,11.012 km,12
7,path 7,9.744 km,20
8,path 8,11.095 km,144
9,path 9,9.681 km,44
10,path 10,11.173 km,13


In [46]:
id = table['score'].idxmax() - 1

m = folium.Map(location=coordinates[0], zoom_start=15)
folium.Marker(nodes_dict[start2]).add_to(m)
folium.Marker(nodes_dict[end2]).add_to(m)

folium.PolyLine(locations=output[id][1], color='blue', tooltip=output[id][2]).add_to(m)
m

In [47]:
def dijkstra(graph, start):
    # Create a dictionary to hold the shortest distances
    distances = {node: float('inf') for node in graph}
    distances[start] = 0
    
    # Create a priority queue to hold the nodes to visit
    pq = [(0, start)]
    
    # Create a dictionary to hold the previous node in the shortest path
    previous = {node: None for node in graph}
    
    # Keep track of visited nodes
    visited = set()
    
    while pq:
        # Get the node with the smallest distance from the start
        (distance, current_node) = heapq.heappop(pq)
        
        # Skip nodes that have already been visited
        if current_node in visited:
            continue
        
        # Mark the current node as visited
        visited.add(current_node)
        
        # Update the distances to the neighbors of the current node
        for (neighbor, cost) in graph[current_node]:
            new_distance = distance + cost
            if new_distance < distances[neighbor]:
                distances[neighbor] = new_distance
                previous[neighbor] = current_node
                heapq.heappush(pq, (new_distance, neighbor))
    
    return previous, distances

In [48]:
prev, dist = dijkstra(nodelist, 36727925)

In [49]:
import copy
def dijkstra_inc(nodelist, start, end, increase, attempts):
    nodecopy = copy.deepcopy(nodelist)
    out = []
    for i in range(attempts):
        prev, dist = dijkstra(nodecopy, start)
        res = []
        path = []
        current = end
        while(current != start):
            res.append(nodes_dict[current])
            path.append(current)
            for i in nodecopy[prev[current]]:
                if i[0] == current:
                    i[1] += increase
            current = prev[current]
        res.append(nodes_dict[start])
        res.reverse()
        path.append(start)
        path.reverse()
        out.append([res, dist[end], path])
    return out
        

In [58]:
fir = dijkstra_inc(nodelist, start, end, 10, 14)

In [59]:
m = folium.Map(location=nodes_dict[start], zoom_start=15)
folium.Marker(nodes_dict[start]).add_to(m)
folium.Marker(nodes_dict[end]).add_to(m)

for i in range(len(fir)):
    folium.PolyLine(locations=fir[i][0], color=colors[i], tooltip=str(round(fir[i][1],3)) + " km").add_to(m)

m

In [60]:
table = pd.DataFrame(columns=['pathNumber', 'distance', 'score'])

nb = 1
for i in fir:
    table.loc[nb] = ['path ' + str(nb), i[1], shortscore(i[2])]
    nb += 1
table

,pathNumber,distance,score
1,path 1,2.666764,132
2,path 2,3.060637,69
3,path 3,3.120028,88
4,path 4,13.587549,75
5,path 5,23.395155,54
6,path 6,24.040969,48
7,path 7,44.668617,63
8,path 8,63.845535,83
9,path 9,64.843980,50
10,path 10,84.614643,37
